## Satellite Image Download from Google Earth Engine using Python

### Import the packages

In [2]:
# import earth engine package and authenticate
import ee
try:
    ee.Initialize()
except:
    ee.Authenticate()
    ee.Initialize()

### Construct the scene ID

For this paper we are using landsat 5 imagery. In this notebook we will downloading an image from 2009 using image id. We'll use the specifc scene ID to get the image. Since the gee api has download size limitations, we export our image to google drive and then download it for our use.

The image id is constructed using following details;

Collection, Processing Level, Landsat Path (Row, Col) and the date.

The image id for the image we are downloading is constructed as follows:

Landsat 5 collection - Level 2 

https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LT05_C02_T1_L2

Landsat Path Row, Col: 145044

Date: 2010-04-28

In [6]:
image_id = 'LANDSAT/LT05/C02/T1_L2/LT05_145044_20100428'


# Load the image
l5_image = ee.Image(image_id)

# Applies scaling factors - as specified in the Landsat 5 Surface Reflectance documentation

def applyScaleFactors(image):
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    return image.addBands(opticalBands, None, True) 


# Apply scaling factors to the image collection
l5_image = applyScaleFactors(l5_image)

# print the metdata
l5_image.getInfo()



{'type': 'Image',
 'bands': [{'id': 'SR_B1',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': -0.2,
    'max': 1.6022125},
   'dimensions': [7811, 6901],
   'crs': 'EPSG:32643',
   'crs_transform': [30, 0, 692085, 0, -30, 2661615]},
  {'id': 'SR_B2',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': -0.2,
    'max': 1.6022125},
   'dimensions': [7811, 6901],
   'crs': 'EPSG:32643',
   'crs_transform': [30, 0, 692085, 0, -30, 2661615]},
  {'id': 'SR_B3',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': -0.2,
    'max': 1.6022125},
   'dimensions': [7811, 6901],
   'crs': 'EPSG:32643',
   'crs_transform': [30, 0, 692085, 0, -30, 2661615]},
  {'id': 'SR_B4',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': -0.2,
    'max': 1.6022125},
   'dimensions': [7811, 6901],
   'crs': 'EPSG:32643',
   'crs_transform': [30, 0, 692085, 0, -30, 2661615]},
  {'id': 'SR_B5',
   'data_type': {'type'

### Select the bands

We select the bands before exporting the image. The bands we are using are: Band 1 (blue), Band 2 (green), Band 3 (red), Band 4 (nir), Band 5 (swir1), Band 7 (swir2). We won't be using Band 6 (thermal) since it's in a different resolution (100m) than the other bands (30m).

In [9]:
#print th bands names
l5_image.bandNames().getInfo()

['SR_B1',
 'SR_B2',
 'SR_B3',
 'SR_B4',
 'SR_B5',
 'SR_B7',
 'SR_ATMOS_OPACITY',
 'SR_CLOUD_QA',
 'ST_B6',
 'ST_ATRAN',
 'ST_CDIST',
 'ST_DRAD',
 'ST_EMIS',
 'ST_EMSD',
 'ST_QA',
 'ST_TRAD',
 'ST_URAD',
 'QA_PIXEL',
 'QA_RADSAT']

In [10]:
# select the bands
l5_bands = ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7']
l5_image = l5_image.select(l5_bands)

### Visualize the image



In [15]:
# Create an interactive map

import geemap

# create a map with cetereda round l5_image  and zoom level 8
Map = geemap.Map()
Map.center_object(l5_image, 8)
# Add the clipped Landsat image to the map
Map.addLayer(l5_image, {'bands': ['SR_B4', 'SR_B3', 'SR_B2'], 'min': 0, 'max': .3}, 'Landsat Image')

# Display the map
Map



Map(center=[23.10755097860265, 78.01956131163406], controls=(WidgetControl(options=['position', 'transparent_b…

### Specify the parameters

In [16]:
# export image specifying scale and region

export_params = {
    'image': l5_image,
    'description': 'LT05_145044_20100428',
    'folder': 'ee',
    'region': l5_image.geometry(),
    'fileFormat' : 'GeoTiff',
    'fileNamePrefix': "LT05_145044_20100428",
     'formatOptions': {
      'cloudOptimized': 'true'
    }
}


## Task the download

In [ ]:

task = ee.batch.Export.image.toDrive(**export_params)
task.start()